In [1]:
!pip install -q faiss-cpu sentence-transformers transformers accelerate pypdf gradio langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 122.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.2/328.2 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 109.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.1/476.1 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
# Loaders de documents
from langchain_community.document_loaders import PyPDFLoader

# Embeddings + Vector DB
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# LLM HuggingFace
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline

# Utilitaires
from pathlib import Path
import gradio as gr

In [3]:
# Dossier contenant vos PDFs
DATA_DIR = "/content/cours"
Path(DATA_DIR).mkdir(parents=True, exist_ok=True)

In [4]:
def load_all_documents(folder):
    """Charge tous les PDFs du dossier."""
    docs = []
    for file in Path(folder).glob("**/*.pdf"):
        loader = PyPDFLoader(str(file))
        docs.extend(loader.load())
    return docs

# Charger les documents
documents = load_all_documents(DATA_DIR)
print(f"📄 {len(documents)} pages chargées")

📄 206 pages chargées


In [5]:
def split_documents(docs, chunk_size=800, overlap=150):
    """Découpe les documents en chunks avec chevauchement."""
    chunks = []

    for doc in docs:
        text = doc.page_content
        start = 0

        while start < len(text):
            chunk_text = text[start:start + chunk_size]
            # Créer un nouveau document avec le même type que l'original
            chunk_doc = type(doc)(
                page_content=chunk_text,
                metadata=doc.metadata
            )
            chunks.append(chunk_doc)
            start += chunk_size - overlap

    return chunks

# Découper les documents
chunks = split_documents(documents)
print(f"✂️ {len(chunks)} chunks créés")

✂️ 176 chunks créés


In [6]:
# Modèle d'embeddings
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

# Création de la base FAISS
vectordb = FAISS.from_documents(chunks, embeddings)
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

print("✅ Base vectorielle créée")

/tmp/ipython-input-168259903.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Base vectorielle créée


In [7]:
LLM_NAME = "bigscience/bloom-560m"

# Charger le tokenizer et le modèle
tokenizer = AutoTokenizer.from_pretrained(LLM_NAME)
model = AutoModelForCausalLM.from_pretrained(LLM_NAME, device_map="auto")

# Créer le pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.3,
    top_p=0.95
)

llm = HuggingFacePipeline(pipeline=pipe)
print("🤖 Modèle chargé")

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Device set to use cuda:0


🤖 Modèle chargé


/tmp/ipython-input-2242607605.py:17: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [11]:
def ask_rag_final(query, retriever, llm):
    """Version finale optimisée."""

    # 1. Récupération
    docs = retriever.invoke(query)
    if isinstance(docs, dict):
        docs = docs.get("documents", [])
    if not docs:
        return "❌ Aucune information trouvée."

    # 2. Contexte court
    context = "\n".join([
        (d.page_content if hasattr(d, "page_content") else str(d))[:700]
        for d in docs[:2]
    ])

    # 3. Prompt ultra-simple
    prompt = f"{context}\n\nQuestion: {query}\nRéponse courte:"

    # 4. Génération
    output = llm.invoke(prompt)
    text = output[0]["generated_text"] if isinstance(output, list) else str(output)

    # 5. Extraction
    if "Réponse courte:" in text:
        text = text.split("Réponse courte:")[-1]

    # Garder 2 premières phrases
    sentences = [s.strip() for s in text.split(".") if len(s) > 15][:2]
    return ". ".join(sentences) + "." if sentences else "Information non disponible."

In [12]:
def chat_interface(user_msg, history):
    """Interface de chat pour Gradio."""
    if not user_msg.strip():
        return history, history

    try:
        answer = ask_rag_final(user_msg, retriever, llm)
    except Exception as e:
        answer = f"❌ Erreur: {str(e)}"

    history.append((user_msg, answer))
    return history, history


# Interface Gradio
with gr.Blocks(title="Chatbot RAG PDF", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 📚 Chatbot RAG - Questions sur vos Documents

    Posez des questions sur vos documents PDF et obtenez des réponses précises basées sur leur contenu.
    """)

    chatbot = gr.Chatbot(
        height=500,
        type="tuples",
        label="Conversation"
    )

    with gr.Row():
        msg_box = gr.Textbox(
            label="Votre question",
            placeholder="Posez votre question ici...",
            scale=4
        )
        submit_btn = gr.Button("Envoyer", scale=1, variant="primary")

    gr.Markdown("💡 **Conseil:** Posez des questions précises pour obtenir les meilleures réponses.")

    # Actions
    msg_box.submit(
        fn=chat_interface,
        inputs=[msg_box, chatbot],
        outputs=[chatbot, chatbot]
    ).then(
        lambda: "",
        outputs=[msg_box]
    )

    submit_btn.click(
        fn=chat_interface,
        inputs=[msg_box, chatbot],
        outputs=[chatbot, chatbot]
    ).then(
        lambda: "",
        outputs=[msg_box]
    )

# Lancer l'interface
demo.launch(share=True, debug=True)

/tmp/ipython-input-2243211113.py:16: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="Chatbot RAG PDF", theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-2243211113.py:23: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-2243211113.py:23: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://215bac553a4f9c7416.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://215bac553a4f9c7416.gradio.live
